In [ ]:
%reload_ext autoreload
%autoreload 2

import time

import numpy as np
from ase.io import read
from ase import Atoms

# Imports specific to this "library"
from pylode import DensityProjectionCalculator
from pylode.utilities.generate_invariants import generate_degree2_invariants, generate_degree2_invariants_from_different

## Generate Density Projection Coefficients

In [ ]:
# Generate a simple data set containing 5 O2 molecules
frames = []
cell = np.eye(3) *12
distances = np.linspace(1.5, 2., 3)
for d in distances:
    positions = [[1,1,1],[1,1,d+1]]
    frame = Atoms('O2', positions=positions, cell=cell, pbc=True)
    frames.append(frame)

# Generate short range (SR) density projection coefficients
# that use a Gaussian density (set potential_exponent = 0 in hypers)
hypers = {
    'smearing':1.5,
    'max_angular':6,
    'max_radial':6,
    'cutoff_radius':5.,
    'potential_exponent':0,
    'radial_basis': 'gto',
    'compute_gradients':False
    }
calculator_sr = DensityProjectionCalculator(**hypers)
calculator_sr.transform(frames)
descriptors_sr = calculator_sr.features

# Generate long range (LR) density projection coefficients
# that use a smeared Coulomb density (set potential_exponent = 1 in hypers)
hypers = {
    'smearing':1.5,
    'max_angular':6,
    'max_radial':1,
    'cutoff_radius':5.,
    'potential_exponent':1,
    'radial_basis': 'monomial',
    'compute_gradients':False
    }
calculator_lr = DensityProjectionCalculator(**hypers)
calculator_lr.transform(frames)
descriptors_lr = calculator_lr.features

## Generate invariants from the projection coefficients

In [ ]:
# SOAP vectors of the structures: obtained by taking quadratic invariants
# combining SR times SR descriptors (rho x rho)
invariants_soap = generate_degree2_invariants(descriptors_sr)

# Pure LODE invariants: obtained by taking quadratic invariants
# combining LR times LR descriptors (V x V)
invariants_lode = generate_degree2_invariants(descriptors_lr)

# Multiscale invariants: combine SR and LR descriptors (rho x V)
invariants_multiscale = generate_degree2_invariants_from_different(descriptors_sr, descriptors_lr)

## Test invariance

In [ ]:
Nrot = 20
np.random.seed(4823131)
from numpy.testing import assert_allclose
for i in range(Nrot):
    # Generate random rotation matrix
    from scipy.linalg import qr
    M = np.random.normal(0,1,(3,3))
    Q, R = qr(M)
    assert_allclose(Q.T@Q, np.eye(3), rtol=1e-15, atol=1e-15)
    # Generate rotated structures
    frames_rotated = []
    for d in distances:
        positions = np.array([[1,1,1],[1,1,d+1]]) @ Q.T
        cell_rot = cell @ Q.T
        frame = Atoms('O2', positions=positions, cell=cell_rot, pbc=True)
        frame.wrap()
        frames_rotated.append(frame)
        
    calculator_rot = DensityProjectionCalculator(**hypers)
    calculator_rot.transform(frames_rotated)
    descriptors_rot = calculator_rot.features
    invariants_rot = generate_degree2_invariants(descriptors_rot)
    
    diff = invariants_lode - invariants_rot
    error = np.linalg.norm(diff) / np.linalg.norm(invariants_lode)
    print(f'Deviation from perfect invariance = {error:4.2e}')
    #if error > 1e-10: print(diff)

## Reshape invariants into input for ML models

In [ ]:
# Reshape invariants into the shape num_environments x num_features
invariants_lode_reshaped = invariants_lode.reshape(len(invariants_lode),-1)